In [16]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [17]:
def make_urllist(date):
    urllist = []
    page_num = 1
    last_urls = set()

    while True:
        url = f'https://finance.naver.com/news/mainnews.naver?date={date}&page={page_num}'
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
        news = requests.get(url, headers=headers)

        # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
        soup = BeautifulSoup(news.content, 'html.parser')

        # CASE 1
        news_list = soup.select('li dl')
        # CASE 2
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))

        current_urls = set()

        # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
        for line in news_list:
            news_url = line.a.get('href')
            current_urls.add(news_url)

        # 현재 페이지의 URL 목록이 이전 페이지와 동일하면 종료합니다.
        if current_urls == last_urls:
            break

        # 중복을 확인한 후에 URL을 리스트에 추가합니다.
        urllist.extend(current_urls)
        last_urls = current_urls
        page_num += 1

    return urllist

In [20]:
original_urls = make_urllist('2024-07-15')
print('뉴스 기사의 개수: ',len(original_urls))

뉴스 기사의 개수:  205


In [21]:
# 변환된 URL을 담을 리스트
url_list = []

for url in original_urls:
    # URL에서 article_id와 office_id 추출
    article_id = url.split('article_id=')[1].split('&')[0]
    office_id = url.split('office_id=')[1].split('&')[0]

    # 새로운 형식의 URL 생성
    new_url = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
    
    # 리스트에 추가
    url_list.append(new_url)

# 결과 출력
print(url_list)

['https://n.news.naver.com/mnews/article/018/0005790845', 'https://n.news.naver.com/mnews/article/215/0001170923', 'https://n.news.naver.com/mnews/article/014/0005213775', 'https://n.news.naver.com/mnews/article/018/0005790832', 'https://n.news.naver.com/mnews/article/009/0005334977', 'https://n.news.naver.com/mnews/article/469/0000812376', 'https://n.news.naver.com/mnews/article/008/0005064050', 'https://n.news.naver.com/mnews/article/015/0005009872', 'https://n.news.naver.com/mnews/article/014/0005213780', 'https://n.news.naver.com/mnews/article/009/0005334930', 'https://n.news.naver.com/mnews/article/029/0002887753', 'https://n.news.naver.com/mnews/article/018/0005790842', 'https://n.news.naver.com/mnews/article/057/0001829868', 'https://n.news.naver.com/mnews/article/009/0005335048', 'https://n.news.naver.com/mnews/article/014/0005213778', 'https://n.news.naver.com/mnews/article/009/0005334928', 'https://n.news.naver.com/mnews/article/009/0005335044', 'https://n.news.naver.com/mnew

In [22]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [23]:
#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code'] = idx2word[str(code)]
  return df

In [24]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [25]:
news_titles = []
news_url =[]
news_contents =[]
news_dates = []

In [26]:
for i in tqdm(url_list):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",len(url_list),'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(url_list)
print("\n[뉴스 내용]")
print(news_contents)

100%|██████████| 205/205 [02:44<00:00,  1.25it/s]

검색된 기사 갯수: 총  205 개

[뉴스 제목]
['[마켓인]CJ라이브시티 백지화…CJ ENM 신용도는 오히려 호재?', '급전 필요한 대주주…한양증권 매각 선회', '"금리 내린다" 해외증시 고공행진… 국내도 수혜주 찾기 분주', '[마켓인]의료파업·부동산PF 여파에…‘알짜’ 한양증권 매각, 예견된 수순?', '"美 생산시설 투자 확대" 기계설비株 미소', '트럼프 피격 사건에 비트코인 3% 급등, 왜?', "'트럼프 피격' 경계심리 받는 코스피…2860선 강보합 마감", 'KB증권, 베스트 증권사 첫 왕좌…"IPO 주관 실적 압도적"', "'떨어지는 칼날' 화장품주 잡는 개미들… '빚투' 더 늘었다", '서울 집값 반등 움직임 … 건설株 꿈틀', '트럼프發 美국채금리 상승… 투자자 비상', "'인천 청라 49층 오피스텔 개발' 대출 1330억…유동화로 차환", '\'억대 소득\' 1020 유튜버 1,300명 "2년 만에 2.5배"', '“금리 내리면 더 뛸거야, 사놓으니 든든”…개미들 거래 40% 늘어난 ‘금’', "'미운오리' 종목의 백조 날갯짓", "브라질 국채 투자 악몽 '스멀스멀'", '“집값 불붙자 콧노래 나오네”…일제히 주가 오른 ‘4종목’', "[투자의 창] 연준 '피벗'에 여전히 신중한 이유", '`트럼프 수혜주`만 급등…코스피, 강보합 마감', '비트코인, 단숨에 6.3만달러 회복…"주말까지 6.5만달러 간다"', "1％만 오르면 또 최고가…다시 뜨는 '귀하신 몸' 금 투자", '국민연금 늘리는 ‘꿀팁’…부자들도 절대 놓치지 않는 ‘이것’은? [노후(NO後) 준비,지금부터⑧]', "우리금융 러브콜에도 버티던 한양증권…전공의 파업에 매각 '백기'", '"이게 밸류업?"알짜 두산밥캣 뺏긴 두산에너 주주 반발', "中 '3중전회' 기대감 업고 항셍테크 ETF 반등", "게임·웹툰 ETF '반토막'…테마 상품 투자주의보", 'LIG넥스원·현대로템 나란히 신고가…방산주 트럼프 모멘텀 지속될까', '최씨 집안의 고려아연 지분 확보에 열 올리는 영풍

In [27]:
df = pd.DataFrame({
    'news_title':news_titles,
    'news_url': url_list,
    'news_contents': news_contents,
    'news_dates': news_dates
})

In [29]:
df.to_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_0715.csv", encoding='utf-8', index=False)